<a href="https://colab.research.google.com/github/TeyrCrimson/tensorflow-projects/blob/master/Plant_seedlings_classification_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
train_dir = '/content/drive/My Drive/Machine Learning/plant-seedlings-classification/train'
validation_dir = '/content/drive/My Drive/Machine Learning/plant-seedlings-classification/test/test'
image_size = 224

In [0]:
import pandas as pd
import os
sample_submission = pd.read_csv('/content/drive/My Drive/Machine Learning/plant-seedlings-classification/sample_submission/sample_submission.csv')

In [0]:
CATEGORIES = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent',
              'Maize', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet']
NUM_CATEGORIES = len(CATEGORIES)

In [6]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense, LeakyReLU
from keras.applications import ResNet50, VGG19
from keras.models import Model, Sequential
from keras.layers import GlobalAveragePooling2D, Conv2D, BatchNormalization, MaxPooling2D
from keras import backend as K
from keras import optimizers
from keras.applications.resnet50 import preprocess_input
from keras.callbacks import ModelCheckpoint
from PIL import Image
import matplotlib.pyplot as plt
import keras
import cv2
import glob
import datetime

Using TensorFlow backend.


In [0]:
def hsvmask(img):
    blur = cv2.GaussianBlur(img, (5,5), 0)
    hsv = cv2.cvtColor(blur, cv2.COLOR_RGB2HSV)
    lower = np.array([45,0,0])
    upper = np.array([95,255,255])
    mask = cv2.inRange(hsv,lower,upper)
    #struc = cv2.getStructuringElement(cv2.MORPH_RECT,(11,11))
    #mask = cv2.morphologyEx(mask,cv2.MORPH_CLOSE,struc)
    #boolean = mask>0
    #boolean = np.dstack([boolean]*3)
    new = np.zeros_like(img,np.uint8)
    new = cv2.bitwise_and(img,img,mask=mask)
    return new

In [0]:
def hsv(img):
    blur = cv2.GaussianBlur(img, (5,5), 0)
    hsv = cv2.cvtColor(blur, cv2.COLOR_RGB2HSV)
    return hsv

In [0]:
def remove_white(img):
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  ret,thresh = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY)
  img[thresh == 255] = 0
  kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
  erosion = cv2.erode(img, kernel, iterations=1)
  return erosion

In [10]:
train = []
y_train1=[]
test = []
train1=[]
validation=[]

for category_id, category in enumerate(CATEGORIES):
    for file in glob.glob(os.path.join('{}/{}/*.png'.format(train_dir,category))):
        train.append([file, category_id, category])
        image = cv2.imread (file)
        image = cv2.resize(remove_white(hsvmask(image)),(image_size,image_size))
        #image = load_img(file, target_size=(200, 200))
        #image = img_to_array(image)
        train1.append (image)
        y_train1.append(category_id)
train = pd.DataFrame(train, columns=['file', 'category_id', 'category'])

for file in glob.glob(os.path.join('{}/*.png'.format(validation_dir))):
        test.append([file])
        image = cv2.imread (file)
        image = cv2.resize(remove_white(hsvmask(image)),(image_size,image_size))
        #image = load_img(file, target_size=(200, 200))
        #image = img_to_array(image)
        validation.append (image)
test = pd.DataFrame(test, columns=['file'])

len(train1)

4750

In [0]:
train_datagen = ImageDataGenerator(rotation_range=360,
                                   zoom_range=0.1,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   horizontal_flip=True,
                                   vertical_flip=True)
    

#test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

#train_generator = train_datagen.flow_from_directory(train_dir,
#                                                   target_size=(image_size, image_size),
#                                                   batch_size=16, 
#                                                   class_mode='categorical',
#                                                   color_mode='rgb',
#                                                   shuffle=True,
#                                                   seed=42)

#validation_generator = test_datagen.flow_from_directory(validation_dir,
#                                                       shuffle=False,
#                                                       target_size=(image_size, image_size),
#                                                       batch_size=16, 
#                                                       class_mode='categorical',
#                                                       color_mode='rgb',
#                                                       seed=42
#                                                       )
#plt.imshow(image)

In [0]:
#files = []
#files.append(os.listdir(f'{train_dir}/Black-grass')[:3])
#files.append(os.listdir(f'{train_dir}/Charlock')[:3])
#files.append(os.listdir(f'{train_dir}/Cleavers')[:3])
#files.append(os.listdir(f'{train_dir}/Common Chickweed')[:3])
#files.append(os.listdir(f'{train_dir}/Common wheat')[:3])
#files.append(os.listdir(f'{train_dir}/Fat Hen')[:3])
#files.append(os.listdir(f'{train_dir}/Loose Silky-bent')[:3])
#files.append(os.listdir(f'{train_dir}/Maize')[:3])
#files.append(os.listdir(f'{train_dir}/Scentless Mayweed')[:3])
#files.append(os.listdir(f'{train_dir}/Shepherds Purse')[:3])
#files.append(os.listdir(f'{train_dir}/Small-flowered Cranesbill')[:3])
#files.append(os.listdir(f'{train_dir}/Sugar beet')[:3])

#print(train_dir + '/' + files[0][0])
#img = plt.imread(os.path.join(train_dir,CATEGORIES[0], files[0][0]))
#plt.imshow(img)
#count = 1
#for i in range(len(files)):
#  for j in range(3):
#    f = plt.figure(figsize=(30,30))
#    img = plt.imread(os.path.join(train_dir,CATEGORIES[i], files[i][j]))
#    f.add_subplot(12,3,count)
#    count = count + 1
#    #plt.imshow(remove_white(hsvmask(img)), aspect=1)
#    plt.imshow(img, aspect=1)

In [0]:
#count = 1
#for i in range(len(files)):
#  for j in range(3):
#    f = plt.figure(figsize=(30,30))
#    img = plt.imread(os.path.join(train_dir,CATEGORIES[i], files[i][j]))
#    f.add_subplot(12,3,count)
#    count = count + 1
#    plt.imshow(hsv(img), aspect=1)
#    #plt.imshow(img, aspect=1)

In [0]:
#count = 1
#for i in range(len(files)):
#  for j in range(3):
#    f = plt.figure(figsize=(30,30))
#    img = plt.imread(os.path.join(train_dir,CATEGORIES[i], files[i][j]))
#    f.add_subplot(12,3,count)
#    count = count + 1
#    plt.imshow(remove_white(hsvmask(img)), aspect=1)
#    #plt.imshow(img, aspect=1)

In [0]:
#files2 = []
#files2.append(os.listdir(f'{validation_dir}')[:20])
#
#print(len(files2[0]))
#
#print(train_dir + '/' + files[0][0])
#img = plt.imread(os.path.join(validation_dir, files2[0][0]))
##plt.imshow(img)
#count = 1
#for i in range(len(files2)):
#  for j in range(20):
#    f = plt.figure(figsize=(30,30))
#    img = plt.imread(os.path.join(validation_dir, files2[i][j]))
#    f.add_subplot(12,3,count)
#    count = count + 1
#    #plt.imshow(img, aspect=1)
#    plt.imshow(remove_white(hsvmask(img)), aspect=1)

In [16]:
#validation_generator
#first_image = validation_generator.next()
#first_image
os.listdir(f'{train_dir}')

['Black-grass',
 'Charlock',
 'Cleavers',
 'Common Chickweed',
 'Common wheat',
 'Fat Hen',
 'Loose Silky-bent',
 'Maize',
 'Scentless Mayweed',
 'Shepherds Purse',
 'Small-flowered Cranesbill',
 'Sugar beet']

In [17]:
#green = np.uint8([[[0,30,30]]])
#hsv_green = cv2.cvtColor(green,cv2.COLOR_BGR2HSV)
#print(hsv_green)
np.array(y_train1).shape

(4750,)

In [35]:
class_size = [len(files) for r, d, files in os.walk(train_dir)][1:]
class_size = [max(class_size) / x for x in class_size]
class_weights = {}
for i in range(len(class_size)):
  class_weights.update({i: class_size[i]})
#[len(files) for r, d, files in os.walk(train_dir)][1:]
class_weights[0] = class_weights[0]*1.5
#class_weights[2] = class_weights[2]*3
#class_weights[4] = class_weights[4]*3
class_weights[6] = class_weights[6]*1.5
#class_weights[8] = class_weights[8]*1.5
class_weights

{0: 3.7300380228136882,
 1: 1.676923076923077,
 2: 2.278745644599303,
 3: 1.0703764320785598,
 4: 2.9592760180995477,
 5: 1.3768421052631579,
 6: 1.5,
 7: 2.9592760180995477,
 8: 1.2674418604651163,
 9: 2.831168831168831,
 10: 1.3185483870967742,
 11: 1.6987012987012986}

In [0]:
#for i in np.random.randint(0, X_train.shape[0], 5):
  #print('\n', y_train[i])
  #plt.imshow(X_train[i])
  #plt.show()

In [0]:
str(datetime.date.today())

filepath = "/content/drive/My Drive/Machine Learning/" + str(datetime.date.today()) + "-weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath,monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train,y_test = train_test_split(np.array(train1),np.array(y_train1), test_size=0.2, shuffle=True, random_state=42)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train/255
X_test = X_test/255
#train_datagen.fit(X_train)
def get_model():
    model = Sequential()
  
    #model.add(Conv2D(filters=32, kernel_size=(3,3), padding='valid', input_shape=X_train[0].shape, activation='relu'))
    #normalize
  
  
    #1st conv
    model.add(Conv2D(filters=32, kernel_size=(3,3), strides=2, padding='same',input_shape=X_train[0].shape))
    #model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    #model.add(LeakyReLU(alpha=0.1))
    model.add(Conv2D(filters=32, kernel_size=(3,3), padding='valid'))
    #model.add(BatchNormalization(axis=3))
    #model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    #model.add(Dropout(0.1))
  
    #2nd conv
    model.add(Conv2D(filters=64, kernel_size=(3,3), strides=2, padding='same'))
    #model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    #model.add(LeakyReLU(alpha=0.1))
    model.add(Conv2D(filters=64, kernel_size=(3,3), padding='valid'))
    #model.add(BatchNormalization(axis=3))
    #model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    #model.add(Dropout(0.1))
    
    #3rd conv
    model.add(Conv2D(filters=128, kernel_size=(3,3), strides=2, padding='same'))
    #model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    #model.add(LeakyReLU(alpha=0.1))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding='valid'))
    #model.add(BatchNormalization(axis=3))
    #model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    #model.add(Dropout(0.1))
    
    #4th conv
    #model.add(Conv2D(filters=256, kernel_size=(3,3), padding='valid', activation='relu'))
    #model.add(LeakyReLU(alpha=0.1))
    #model.add(BatchNormalization(axis=3))
    #model.add(LeakyReLU(alpha=0.1))
    #model.add(Conv2D(filters=256, kernel_size=(3,3), padding='valid', activation='relu'))
    #model.add(BatchNormalization(axis=3))
    #model.add(LeakyReLU(alpha=0.1))
    #model.add(MaxPooling2D(pool_size=(2,2)))
    #model.add(BatchNormalization(axis=3))
    #model.add(Dropout(0.1))
  
    #flatten
    model.add(GlobalAveragePooling2D())
    #model.add(Flatten())
  
    #dense layer
    model.add(Dense(units=128, activation='tanh'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(units=128, activation='tanh'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))      
    model.add(Dense(units=12, activation='softmax'))
  
    #dropout
    #model.add(Dropout(0.3))
  
    return model
  
# test model
epochs = 50
learning_rate = 0.001
decay_rate = learning_rate / epochs
momentum = 0.9
sgd = keras.optimizers.SGD(lr=learning_rate, momentum=momentum, nesterov=True)
adam = keras.optimizers.Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
model_sgd = get_model()
model_sgd.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 112, 112, 32)      896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 112, 112, 32)      128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 110, 110, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 55, 55, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 55, 55, 32)        128       
_________________________________________________________________
conv2d_3 (Conv2D)    

In [0]:
from keras import metrics
X_train, X_test, y_train,y_test = train_test_split(np.array(train1),np.array(y_train1), test_size=0.2, shuffle=True, random_state=42)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train/255
X_test = X_test/255

In [36]:
model_sgd.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['acc'])
#history_sgd = model_sgd.fit(X_train,y_train, batch_size=16, epochs=60, validation_data=(X_test,y_test), shuffle=True, callbacks=callbacks_list)
#model_sgd.load_weights('/content/drive/My Drive/Machine Learning/2019-04-10-weights-improvement-02-0.52.hdf5')
history_sgd = model_sgd.fit_generator(train_datagen.flow(X_train, y_train, batch_size=16),
                                     validation_data=(X_test,y_test),
                                     steps_per_epoch=X_train.shape[0]//16,
                                     epochs=10,
                                     callbacks=callbacks_list,
                                     class_weight = class_weights,
                                     verbose=1,
                                     shuffle=True)

Epoch 1/10
237/237 [==============================] - 35s 147ms/step - loss: 0.4145 - acc: 0.9130 - val_loss: 0.2659 - val_acc: 0.9116

Epoch 00001: val_acc did not improve from 0.91158
Epoch 2/10
237/237 [==============================] - 35s 149ms/step - loss: 0.4402 - acc: 0.9159 - val_loss: 0.2793 - val_acc: 0.8947

Epoch 00002: val_acc did not improve from 0.91158
Epoch 3/10
237/237 [==============================] - 36s 150ms/step - loss: 0.4329 - acc: 0.9106 - val_loss: 0.2673 - val_acc: 0.9095

Epoch 00003: val_acc did not improve from 0.91158
Epoch 4/10
237/237 [==============================] - 35s 149ms/step - loss: 0.4369 - acc: 0.9095 - val_loss: 0.2632 - val_acc: 0.9095

Epoch 00004: val_acc did not improve from 0.91158
Epoch 5/10
237/237 [==============================] - 34s 144ms/step - loss: 0.4243 - acc: 0.9153 - val_loss: 0.2746 - val_acc: 0.9042

Epoch 00005: val_acc did not improve from 0.91158
Epoch 6/10
237/237 [==============================] - 35s 147ms/step -

In [0]:
model_sgd.load_weights('/content/drive/My Drive/Machine Learning/absolutebest.hdf5')

In [37]:
from sklearn.metrics import confusion_matrix
# PREDICTIONS
y_pred = model_sgd.predict(X_test)
#print(y_pred)
y_class = np.argmax(y_pred, axis=1)
#print(y_class)
#print(y_test)
y_check = y_test
#print(y_check)
cmatrix = confusion_matrix(y_check, y_class)
print(cmatrix)

[[ 58   0   0   0   2   0   7   0   0   0   0   0]
 [  0  79   0   0   0   0   0   0   0   0   0   0]
 [  0   1  54   1   0   0   0   0   0   0   1   0]
 [  1   0   2 114   0   0   2   1   0   1   0   4]
 [  1   0   0   0  34   0   0   0   0   0   0   0]
 [  0   0   1   1   4  73   0   0   0   0   0   1]
 [ 36   0   0   0   2   0 101   0   1   0   0   0]
 [  0   0   0   0   0   0   0  45   0   0   0   1]
 [  2   0   0   3   0   0   0   0  87   1   0   1]
 [  0   1   0   3   0   0   0   0   1  37   0   0]
 [  0   1   0   0   0   0   0   1   0   0  99   0]
 [  0   1   0   0   0   0   0   2   0   0   0  81]]


In [0]:
retrain_dir1 = '/content/drive/My Drive/Machine Learning/plant-seedlings-classification/train/' + CATEGORIES[0]
retrain_dir2 = '/content/drive/My Drive/Machine Learning/plant-seedlings-classification/train/' + CATEGORIES[6]

In [0]:
retrain = []
retrain1 = []
y_retrain = []

for file in glob.glob(os.path.join('{}/*.png'.format(retrain_dir1))):
    retrain.append([file, '0', CATEGORIES[0]])
    image = cv2.imread (file)
    image = cv2.resize(remove_white(hsvmask(image)),(200,200))
    #image = load_img(file, target_size=(200, 200))
    #image = img_to_array(image)
    retrain1.append(image)
    y_retrain.append('0')
for file in glob.glob(os.path.join('{}/*.png'.format(retrain_dir2))):
    retrain.append([file, '6', CATEGORIES[6]])
    image = cv2.imread (file)
    image = cv2.resize(remove_white(hsvmask(image)),(200,200))
    #image = load_img(file, target_size=(200, 200))
    #image = img_to_array(image)
    retrain1.append(image)
    y_retrain.append('6')
retrain = pd.DataFrame(retrain, columns=['file', 'category_id', 'category'])

In [0]:
[len(files) for r, d, files in os.walk(train_dir)]

[0, 263, 390, 287, 611, 221, 475, 654, 221, 516, 231, 496, 385]

In [25]:
for layer in model_sgd.layers:
  layer.trainable = True
for layer in model_sgd.layers[:-7]:
  layer.trainable = False

for layer in model_sgd.layers:
  print(layer, layer.trainable)

<keras.layers.convolutional.Conv2D object at 0x7f2d0f0d7eb8> False
<keras.layers.normalization.BatchNormalization object at 0x7f2d0f0e6358> False
<keras.layers.convolutional.Conv2D object at 0x7f2d0f0e62e8> False
<keras.layers.pooling.MaxPooling2D object at 0x7f2d0f0e6400> False
<keras.layers.normalization.BatchNormalization object at 0x7f2d005b8f98> False
<keras.layers.convolutional.Conv2D object at 0x7f2d005b8978> False
<keras.layers.normalization.BatchNormalization object at 0x7f2d0045c8d0> False
<keras.layers.convolutional.Conv2D object at 0x7f2d004bde80> False
<keras.layers.pooling.MaxPooling2D object at 0x7f2d00368a20> False
<keras.layers.normalization.BatchNormalization object at 0x7f2d003c8b00> False
<keras.layers.convolutional.Conv2D object at 0x7f2d002fbd68> False
<keras.layers.normalization.BatchNormalization object at 0x7f2d002ac8d0> False
<keras.layers.convolutional.Conv2D object at 0x7f2d0026ad68> False
<keras.layers.pooling.MaxPooling2D object at 0x7f2d001b6828> False
<k

In [0]:
X_train2, X_test2, y_train2,y_test2 = train_test_split(np.array(retrain1),np.array(y_retrain), test_size=0.2, random_state=42)
X_train2 = X_train2.astype('float32')
X_test2 = X_test2.astype('float32')
X_train2 = X_train2/255
X_test2 = X_test2/255

In [0]:
model_sgd.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [0]:
history_sgd = model_sgd.fit_generator(train_datagen.flow(X_train2, y_train2, batch_size=16),
                                     validation_data=(X_test2,y_test2),
                                     epochs=1,
                                     steps_per_epoch=X_train2.shape[0]//16,
                                     callbacks=callbacks_list,
                                     verbose=1,
                                     shuffle=True)

Epoch 1/1
45/45 [==============================] - 8s 181ms/step - loss: 3.9773 - acc: 0.2250 - val_loss: 1.0680 - val_acc: 0.7228

Epoch 00001: val_acc did not improve from 0.86421


In [0]:
import csv
from google.colab import files

savePath = '/content/drive/My Drive/Machine Learning/'

#Y_new = model_sgd.predict_classes(validation)
with open(str(savePath)+'submission.csv','w') as f1:
  writer = csv.writer(f1, delimiter=',', lineterminator='\n')
  #print(validation[0])
  #print(Y_new[0])
  writer.writerow(['file','species'])
  for i in range(len(validation)):
    xid = test.iloc[i]['file'].replace(validation_dir, '')[1:]
    data = CATEGORIES[model_sgd.predict_classes(np.expand_dims(validation[i]/255, axis=0))[0]]
    writer.writerow([xid, data])
  #files.download(str(savePath)+'submission.csv')

## The Learning Rate

The learning rate is an hyper-parameter that controls how much we are  adjusting the weights of our network with respect to the loss gradient. 

The lower this value the slower the travel along the downward slope, if the value is higher the contrary will happen. Even though having a low value might sounds like a good idea, so we make sure we don't miss the minima loss value it would also mean that we can get stuck in a local minima easier and we'll take a longer time to converge.

The following formula shows how the learning rate affects the changes in the weights during the learning process:

``new_weight = existing_weight — learning_rate * gradient``

### Stochastic gradient descent(SGD)

In keras SGD learning comes with Nesterov enabled by defult. Nesterov accelerated grading is a different version of the momentum method which has stronger theoretical converge guarantees for convex functions. 

![Nesterov](Notebook/Nesterov.png)

**Good for:**
* Shallow networks

In [0]:
keras.optimizers.SGD(lr=10**-3, nesterov=True)

#### Time-based decay

The SGD optimizer takes decay and lr arguments and updates the learning rate by a decreasing factor in each epoch.

#### Momentum

Momentum is another argument in SGD optimizer which we could tweak to obtain faster convergence. Unlike classical SGD, momentum method helps the parameter vector to build up velocity in any direction with constant gradient descent so as to prevent oscillations. A typical choice of momentum is between 0.5 to 0.9.

In [0]:
epochs = 10
learning_rate = 0.001
decay_rate = learning_rate / epochs
momentum = 0.9
sgd = keras.optimizers.SGD(lr=learning_rate, momentum=momentum, nesterov=False)

In [0]:
model_sgd = get_model()
model_sgd.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 198, 198, 32)      896       
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 196, 196, 32)      9248      
_________________________________________________________________
batch_normalization_10 (Batc (None, 196, 196, 32)      128       
_________________________________________________________________
dropout_10 (Dropout)         (None, 196, 196, 32)      0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 98, 98, 32)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 96, 96, 32)        9248      
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 94, 94, 32)        9248      
__________

In [0]:
model_sgd.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.SGD(), metrics=['accuracy'])

In [0]:
history_sgd = model_sgd.fit(X_train,y_train, batch_size=16, epochs=60, validation_data=(X_test,y_test), shuffle=True)
#np.array(train1)

Train on 2138 samples, validate on 535 samples
Epoch 1/60
2138/2138 [==============================] - 19s 9ms/step - loss: 1.3630 - acc: 0.5042 - val_loss: 3.2443 - val_acc: 0.3551
Epoch 2/60
2138/2138 [==============================] - 18s 8ms/step - loss: 0.7050 - acc: 0.7404 - val_loss: 1.1600 - val_acc: 0.6187
Epoch 3/60
2138/2138 [==============================] - 18s 8ms/step - loss: 0.4437 - acc: 0.8358 - val_loss: 9.2388 - val_acc: 0.1682
Epoch 4/60
2138/2138 [==============================] - 17s 8ms/step - loss: 0.2426 - acc: 0.9167 - val_loss: 6.0546 - val_acc: 0.2486
Epoch 5/60
2138/2138 [==============================] - 17s 8ms/step - loss: 0.1406 - acc: 0.9509 - val_loss: 0.8563 - val_acc: 0.7140
Epoch 6/60
2138/2138 [==============================] - 17s 8ms/step - loss: 0.0945 - acc: 0.9724 - val_loss: 1.2947 - val_acc: 0.6374
Epoch 7/60
2138/2138 [==============================] - 18s 8ms/step - loss: 0.1232 - acc: 0.9654 - val_loss: 2.0899 - val_acc: 0.4542
Epoch 8/

In [0]:
# Plot training & validation accuracy values
plt.plot(history_sgd.history['acc'])
plt.plot(history_sgd.history['val_acc'])
plt.title('SGD accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history_sgd.history['loss'])
plt.plot(history_sgd.history['val_loss'])
plt.title('SGD loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

## RMSprop

It is an unpublished, adaptive learning rate method proposed by Geoff Hinton in Lecture 6e of his Coursera Class.

Rather than having just a global, scalar learning rate, we have a vector of learning rates for each trainable parameter. It is iteratively updated with a running average of magnitudes of squares of previous gradients. Changes to the weights during training are now not purely in the direction of the gradient, but rather in the direction of the elementwise division of the gradient by this vector you are maintaining. 

In [0]:
keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)

In [0]:
model_rmsprop = get_model()
model_rmsprop.summary()

In [0]:
model_rmsprop.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.RMSprop(), metrics=['accuracy'])

In [0]:
history_rmsprop = model_rmsprop.fit(X_train, y_train, batch_size=16, epochs=60, validation_data=(X_test, y_test), shuffle=True)

In [0]:
# Plot training & validation accuracy values
plt.plot(history_rmsprop.history['acc'])
plt.plot(history_rmsprop.history['val_acc'])
plt.title('RMSprop accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history_rmsprop.history['loss'])
plt.plot(history_rmsprop.history['val_loss'])
plt.title('RMSprop loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

## Adam

Adaptive moment estimation

Adam = RMSprop + Momentum

**Good for:**

* Relatively low memory requirements (though higher than gradient descent and gradient descent with momentum)  
* Usually works well even with little tuning of hyperparameters.

In Keras, we can define it like this:

In [0]:
keras.optimizers.Adam(lr=0.001)

In [0]:
model_adam = get_model()
model_adam.summary()

In [0]:
model_adam.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

In [0]:
history_adam = model_adam.fit(X_train, y_train, batch_size=16, epochs=60, validation_data=(X_test, y_test), shuffle=True)

In [0]:
# Plot training & validation accuracy values
plt.plot(history_adam.history['acc'])
plt.plot(history_adam.history['val_acc'])
plt.title('Adam accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history_adam.history['loss'])
plt.plot(history_adam.history['val_loss'])
plt.title('Adam loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

## Adagrad

It makes big updates for infrequent parameters and small updates for frequent parameters. For this reason, it is well-suited for dealing with sparse data.

The main benefit of Adagrad is that we don’t need to tune the learning rate manually. Most implementations use a default value of 0.01 and leave it at that.

**Disadvantage:**

Its main weakness is that its learning rate is always Decreasing and decaying.

In [0]:
keras.optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)

In [0]:
model_adagrad = get_model()
model_adagrad.summary()

In [0]:
model_adagrad.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adagrad(), metrics=['accuracy'])

In [0]:
history_adagrad = model_adagrad.fit(X_train, y_train, batch_size=128, epochs=60, validation_data=(X_test, y_test), shuffle=True)

In [0]:
# Plot training & validation accuracy values
plt.plot(history_adagrad.history['acc'])
plt.plot(history_adagrad.history['val_acc'])
plt.title('Adagrad accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history_adagrad.history['loss'])
plt.plot(history_adagrad.history['val_loss'])
plt.title('Adagrad loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

## AdaDelta

More robust extension of Adagrad that adapts learning rates based on a moving window of gradient updates, instead of accumulating all past gradients. This way, Adadelta continues learning even when many updates have been done. Compared to Adagrad, in the original version of Adadelta you don't have to set an initial learning rate. In this version, initial learning rate and decay factor can be set, as in most other Keras optimizers.

It is recommended to leave the parameters of this optimizer at their default values.

In [0]:
keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)

In [0]:
model_adadelta = get_model()
model_adadelta.summary()

In [0]:
model_adadelta.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

In [0]:
history_adadelta = model_adadelta.fit(X_train, y_train, batch_size=128, epochs=60, validation_data=(X_test, y_test), shuffle=True)

In [0]:
# Plot training & validation accuracy values
plt.plot(history_adadelta.history['acc'])
plt.plot(history_adadelta.history['val_acc'])
plt.title('Adadelta accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history_adadelta.history['loss'])
plt.plot(history_adadelta.history['val_loss'])
plt.title('Adadelta loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

<img src="Notebook/optimizer.gif" alt="optimizers">

# General Plot

In [0]:
plt.plot(history_sgd.history['acc'])
plt.plot(history_adam.history['acc'])
plt.plot(history_rmsprop.history['acc'])
plt.plot(history_adadelta.history['acc'])
plt.plot(history_adagrad.history['acc'])

plt.title('Accuracies')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad'], loc='upper left')
plt.show()

In [0]:
import csv

savePath = '/content/drive/My Drive/Machine Learning/'

#Y_new = model_sgd.predict_classes(validation)
with open(str(savePath)+'submission.csv','w') as f1:
  writer = csv.writer(f1, delimiter=',', lineterminator='\n')
  #print(validation[0])
  #print(Y_new[0])
  for i in range(len(validation)):
    xid = glob.glob(os.path.join('{}/*.png'.format(validation_dir)))[i].replace(validation_dir, '')[1:]
    data = CATEGORIES[model_sgd.predict_classes(np.expand_dims(validation[i], axis=0))[0]]
    writer.writerow([xid, data])